In [1]:
import os
import pandas as pd
from llama_index.llms import AzureOpenAI
from llama_index.embeddings import AzureOpenAIEmbedding
from langchain.embeddings import HuggingFaceEmbeddings
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext, StorageContext, load_index_from_storage, set_global_service_context
import logging
import sys
from dotenv import load_dotenv
from llama_index.llms import Ollama
import numpy as np
from trulens_eval import TruLlama, Feedback, Tru, feedback
from langchain.llms import Ollama
from trulens_eval.tru_custom_app import instrument
from trulens_eval import LiteLLM
import litellm
litellm.set_verbose=False
tru = Tru()

load_dotenv('/Users/jeana/.env')

logging.basicConfig(
    stream=sys.stdout, level=logging.WARNING
)  # logging.DEBUG for more verbose output
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.


### Input and Parameters

In [2]:
input_file_location = [r"/Users/jeana/Retrieval-Augmented-Generation/LlamaIndex/paul_graham_essay.txt"]

### Define Exceptions

In [3]:
class modelException(Exception):
    def __init__(self, invalid_value, allowed_values):
        self.invalid_value = invalid_value
        self.allowed_values = allowed_values
        message = f"Invalid value: {invalid_value}. Allowed values are: {', '.join(allowed_values)}"
        super().__init__(message)

### Define Evaluator with Feedback functions

In [4]:
def feedbacks(llm_name):

    ##### INITIALZE FEEDBACK FUNCTION(S)#######
    if llm_name == 'azureoai':
        # Initialize AzureOpenAI-based feedback function collection class:
        llm_provider = feedback.AzureOpenAI(
                                        deployment_name=os.environ['OPENAI_DEPLOYMENT_NAME'],
                                        api_key = os.environ['OPENAI_API_KEY'],
                                        api_version=os.environ['OPENAI_DEPLOYMENT_VERSION'],
                                        azure_endpoint=os.environ['OPENAI_DEPLOYMENT_ENDPOINT'],
                                        # model = os.environ['OPENAI_MODEL_NAME']
                                        )
    elif llm_name == 'llama2':
        llm_provider = LiteLLM(model_engine="ollama/llama2", api_base='http://localhost:11434')

    # Question/answer relevance between overall question and answer.
    f_qa_relevance = Feedback(llm_provider.relevance, name = "Answer Relevance").on_input_output()

    # Question/statement relevance between question and each context chunk.
    f_qs_relevance = Feedback(llm_provider.qs_relevance, name = "Context Relevance").on_input().on(
        TruLlama.select_source_nodes().node.text
    ).aggregate(np.mean)

    # groundedness of output on the context
    groundedness = feedback.Groundedness(
                    # summarize_provider=azopenai, 
                    groundedness_provider=llm_provider)
    f_groundedness = Feedback(groundedness.groundedness_measure, name = "Groundedness").on(TruLlama.select_source_nodes().node.text).on_output()
    
    feedbacks=[f_groundedness, f_qa_relevance, f_qs_relevance]
    
    return feedbacks


In [5]:
feedbacks = feedbacks('llama2')

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input statement will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


### Load Documents

In [6]:
from llama_index import download_loader

ArxivReader = download_loader("ArxivReader")

loader = ArxivReader()
documents = loader.load_data(search_query='id:2311.12399', max_results=1)

In [7]:
documents

[Document(id_='1af39076-786a-4914-9ec7-812adc528ca8', embedding=None, metadata={'page_label': '1', 'file_name': '25b4824683c0f3870d9744973d6258bd.pdf', 'Title of this paper': 'A Survey of Graph Meets Large Language Model: Progress and Future Directions', 'Authors': 'Yuhan Li, Zhixun Li, Peisong Wang, Jia Li, Xiangguo Sun, Hong Cheng, Jeffrey Xu Yu', 'Date published': '11/21/2023', 'URL': 'http://arxiv.org/abs/2311.12399v2'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='f07e4085285761d5e7970ae398933808400f7e103ea08bba887d9c735e7f1314', text='A Survey of Graph Meets Large Language Model: Progress and Future Directions\nYuhan Li1∗,Zhixun Li2∗,Peisong Wang3∗,Jia Li1†,Xiangguo Sun2,\nHong Cheng2,Jeffrey Xu Yu2\n1The Hong Kong University of Science and Technology (Guangzhou)\n2The Chinese University of Hong Kong\n3Tsinghua University\nAbstract\nGraph plays a significant role in representing and\nanalyzing complex relationships in real-world appli-\n

### RAG Class

In [8]:
class RAG():
    llm_list = ['llama2', 'gpt3.5Turbo']
    embedding_list = ['text-embedding-ada-002', 'sentence-transformers/all-mpnet-base-v2', "BAAI/bge-small-en-v1.5"]

    def __init__(self, llm_name, embedding_model, documents, feedbacks):

        self.feedbacks = feedbacks
           
        if llm_name not in self.llm_list:
            raise modelException(llm_name, self.llm_list)
        if embedding_model not in self.embedding_list:
            raise modelException(embedding_model, self.embedding_list)        

        ## Getting the LLM for prediction
        if llm_name == 'gpt3.5Turbo':
            self.llm = AzureOpenAI(
                    # model= os.environ['OPENAI_MODEL_NAME'],
                    model = llm_name,
                    deployment_name= os.environ['OPENAI_DEPLOYMENT_NAME'],
                    api_key=os.environ['OPENAI_API_KEY'],
                    azure_endpoint=os.environ['OPENAI_DEPLOYMENT_ENDPOINT'],
                    api_version=os.environ['OPENAI_DEPLOYMENT_VERSION'],
                )
        elif llm_name == 'llama2':
            self.llm = Ollama(model="llama2")
        
        ## Gettting the embedding model
        if embedding_model == 'text-embedding-ada-002':
            self.embedding_model = AzureOpenAIEmbedding(
                    # model=os.environ['OPENAI_EMBEDDING_MODEL_NAME'],
                    model=embedding_model,
                    deployment_name=os.environ['OPENAI_EMBEDDING_DEPLOYMENT_NAME'],
                    api_key=os.environ['OPENAI_API_KEY'],
                    azure_endpoint=os.environ['OPENAI_DEPLOYMENT_ENDPOINT'],
                    api_version=os.environ['OPENAI_DEPLOYMENT_VERSION'],
                )
        elif embedding_model in ['sentence-transformers/all-mpnet-base-v2', "BAAI/bge-small-en-v1.5"]:
            self.embedding_model = HuggingFaceEmbeddings(
                    model_name=embedding_model)
        
        #Set service context, documents and Index
        self.service_context = ServiceContext.from_defaults(embed_model=self.embedding_model, llm = self.llm)

        set_global_service_context(self.service_context)

        # check if storage already exists (under the specific embedding model)
        if not os.path.exists("./storage/" + embedding_model):
            # load the documents and create the index
            self.documents = documents
            self.index = VectorStoreIndex.from_documents(self.documents, service_context=self.service_context)
            # store it for later
            self.index.storage_context.persist("./storage/" + embedding_model)
            self.storage_context = None
        else:
            # load the existing index from the specific embedding model
            self.storage_context = StorageContext.from_defaults(persist_dir="./storage/" + embedding_model)
            self.index = load_index_from_storage(self.storage_context)

        # self.documents = SimpleDirectoryReader(
        #             input_files=[r"/Users/jeana/Retrieval-Augmented-Generation/LlamaIndex/paul_graham_essay.txt"] #or just indicate the fullpath of the folder containing the data
        #                         ).load_data()
        # self.index = VectorStoreIndex.from_documents(self.documents, service_context=self.service_context)

    def query(self, query: str) -> str:
        ### INSTRUMENT CHAIN FOR LOGGING WITH TRULENS

        query_engine = self.index.as_query_engine()

        tru_query_engine_recorder = TruLlama(query_engine,
                app_id='LlamaIndex_App1',
                feedbacks=self.feedbacks)

        with tru_query_engine_recorder as recorder:
            answer = query_engine.query(query)
            print(answer.get_formatted_sources())
            print("query was:", query)
            print("answer was:", answer)

# Initialize and Run Queries

In [9]:
llm_name =  'llama2' #gpt3.5Turbo , llama2
embedding_model = 'sentence-transformers/all-mpnet-base-v2' # text-embedding-ada-002 , sentence-transformers/all-mpnet-base-v2, local
rag = RAG(llm_name, embedding_model, documents, feedbacks)

In [11]:
rag.query('How does LLM help graph-related tasks?')

> Source (Doc id: 04460240-1298-4c8c-a62f-d90ce7e2e85a): (a) Flatten-based(b) GNN-basedText Attributes
GraphStructureGNN
InitialFeaturesGraphEmbeddingsLLM...

> Source (Doc id: 0eb0befb-c740-44ee-8623-ce0a89448b4c): , 2023a] further set the decoding
temperature of the LLM to 0, in order to reduce the variance
of...
query was: How does LLM help graph-related tasks?
answer was: LLM can assist graph-related tasks in several ways:

1. Graph structure representation: LLMs can flatten the graph structure into a sequence of nodes or tokens, which facilitates direct processing of graph data by LLMs through text sequences.
2. Prediction: LLMs can make predictions for a wide range of graph-related tasks, such as classifications and reasonings, within a unified generative paradigm.
3. Graph representation enhancement: Some methods, such as GIMLET [Zhao et al. , 2023a] and InstructGLM [Ye et al. , 2023], fine-tune LLMs to directly output predicted labels, empowering them to provide accurate predic

## TruLens Evaluation Metrics

In [ ]:
#View results in notebook
tru.get_records_and_feedback(app_ids=[])[0] # pass an empty list of app_ids to get all

In [19]:
#show only input, output, groundness, answer and context scores
records, feedback = tru.get_records_and_feedback(app_ids=[])
pd.set_option("display.max_colwidth", None)
records[["input", "output"] + feedback]

,input,output,Context Relevance,Groundedness:::full_doc_score,Answer Relevance
0,"""What is most interesting about this essay?""","""The most interesting aspect of this essay is the author's reflection on how his choices and experiences have been shaped by his desire to work on things that are not prestigious, but instead genuinely interest him. He notes that when he finds himself drawn to something despite its lack of prestige, it's a sign that there's something real to be discovered there, and that he has the right kind of motives. He also shares how he and his friends have started their own investment firm, after becoming frustrated with the slow decision-making of venture capital firms.\n\nOverall, the essay provides a personal and introspective look at the author's approach to work and life, and how he prioritizes following his interests and passions over conventional expectations of success.""",0.20,0.0,0.8
1,"""What is most interesting about this essay?""","""Based on the provided context, the most interesting aspect of this essay is the author's reflection on how they have chosen what to work on throughout their life. The author notes that they have always been drawn to working on things that were not prestigious at the time, such as still life painting, Viaweb, and Y Combinator. They also observe that when they find themselves interested in something despite its lack of prestige, it is a sign that there is something real to be discovered and that they have the right kind of motives. The author also shares personal anecdotes and experiences, such as their dinner parties and meeting Jessica Livingston, which led to them starting an investment firm together. Overall, the essay provides insight into the author's thought process and approach to work, and how they have managed to balance creativity and practicality throughout their career.""",1.00,0.9,1.0
2,"""What is most interesting about this essay?""","""Based on the context information provided, the most interesting aspect of this essay is the author's personal journey and experiences that have shaped their perspective on life. The author reflects on how they have chosen what to work on in the past, and how their approach to life has evolved over time. They share stories about their experiences, such as moving to England with their family, starting a business, and meeting new people, which demonstrate their ability to adapt and learn from their surroundings.\n\nThe author also touches on various topics, including the nature of work, the importance of having the right motives, and the value of perseverance. They offer insights into the world of technology and entrepreneurship, drawing from their own experiences in these fields.\n\nOverall, the essay provides a personal and introspective look at the author's life, exploring themes of growth, learning, and self-discovery.""",0.85,1.0,0.9
3,"""What is most interesting about this essay?""","""Based on the context information provided, the most interesting aspect of this essay is the author's personal experiences and reflections on various topics such as:\n\n1. The evolution of computers and how the author skipped a step in the process.\n2. The differences between Italian and English words, and how the author used this knowledge to create more Italian phrases.\n3. The author's experiences living in different countries and cities, including England and Florence.\n4. The author's thoughts on the importance of working on things that are not prestigious, and how this has led to discovering something real and having the right kind of motives.\n5. The author's personal life, including their Thursday dinner parties and their relationship with Jessica Livingston.\n6. The author's thoughts on venture capital and angel investing, and how they decided to start their own investment firm.\n\nThese experiences and reflections provide a unique perspective on the author's life and interests, making the essay interesting to read.""",1.00,1.0

In [20]:
tru.get_leaderboard(app_ids=[])

,Context Relevance,Groundedness:::full_doc_score,Answer Relevance,latency,total_cost
app_id,,,,,
LlamaIndex_App1,0.78125,0.85,0.585714,26.142857,0.0


In [22]:
#view results in dashboard
tru.run_dashboard() # open a local streamlit app to explore

# tru.stop_dashboard() # stop if needed

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

### Load app

In [ ]:
def load_llamaindex_app():
    
    import os
    from llama_index.llms import AzureOpenAI
    from llama_index.embeddings import AzureOpenAIEmbedding
    from langchain.embeddings import HuggingFaceEmbeddings
    from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext, StorageContext, load_index_from_storage, set_global_service_context
    from dotenv import load_dotenv
    from llama_index.llms import Ollama
    from langchain.llms import Ollama
    import litellm
    litellm.set_verbose=False

    load_dotenv('/Users/jeana/.env')

    # input_file_location = [r"/Users/jeana/Retrieval-Augmented-Generation/LlamaIndex/paul_graham_essay.txt"]

    ArxivReader = download_loader("ArxivReader")

    loader = ArxivReader()
    documents = loader.load_data(papers_dir = r'/Users/jeana/Retrieval-Augmented-Generation/.papers'
                                , search_query='id:2311.12399', max_results=1)

    class modelException(Exception):
        def __init__(self, invalid_value, allowed_values):
            self.invalid_value = invalid_value
            self.allowed_values = allowed_values
            message = f"Invalid value: {invalid_value}. Allowed values are: {', '.join(allowed_values)}"
            super().__init__(message)

    llm_list = ['llama2', 'gpt3.5Turbo']
    embedding_list = ['text-embedding-ada-002', 'sentence-transformers/all-mpnet-base-v2', "BAAI/bge-small-en-v1.5"]
    
    llm_name =  'llama2' #gpt3.5Turbo , llama2
    embedding_model_name = 'sentence-transformers/all-mpnet-base-v2' # text-embedding-ada-002 , sentence-transformers/all-mpnet-base-v2, local

    if llm_name not in llm_list:
        raise modelException(llm_name, llm_list)
    if embedding_model_name not in embedding_list:
        raise modelException(embedding_model_name, embedding_list)        

    ## Getting the LLM for prediction
    if llm_name == 'gpt3.5Turbo':
        llm = AzureOpenAI(
                # model= os.environ['OPENAI_MODEL_NAME'],
                model = llm_name,
                deployment_name= os.environ['OPENAI_DEPLOYMENT_NAME'],
                api_key=os.environ['OPENAI_API_KEY'],
                azure_endpoint=os.environ['OPENAI_DEPLOYMENT_ENDPOINT'],
                api_version=os.environ['OPENAI_DEPLOYMENT_VERSION'],
            )
    elif llm_name == 'llama2':
        llm = Ollama(model="llama2")

    ## Gettting the embedding model
    if embedding_model_name == 'text-embedding-ada-002':
        embedding_model = AzureOpenAIEmbedding(
                # model=os.environ['OPENAI_EMBEDDING_MODEL_NAME'],
                model=embedding_model_name,
                deployment_name=os.environ['OPENAI_EMBEDDING_DEPLOYMENT_NAME'],
                api_key=os.environ['OPENAI_API_KEY'],
                azure_endpoint=os.environ['OPENAI_DEPLOYMENT_ENDPOINT'],
                api_version=os.environ['OPENAI_DEPLOYMENT_VERSION'],
            )
    elif embedding_model_name in ['sentence-transformers/all-mpnet-base-v2', "BAAI/bge-small-en-v1.5"]:
        embedding_model = HuggingFaceEmbeddings(model_name=embedding_model_name)

    #Set service context, documents and Index
    service_context = ServiceContext.from_defaults(embed_model=embedding_model, llm = llm)

    set_global_service_context(service_context)

    # check if storage already exists (under the specific embedding model)
    if not os.path.exists("./storage/" + embedding_model_name):
        # load the documents and create the index
        # documents = SimpleDirectoryReader(input_files=[r"/Users/jeana/Retrieval-Augmented-Generation/LlamaIndex/paul_graham_essay.txt"] #or just indicate the fullpath of the folder containing the data
        #                 ).load_data()
        index = VectorStoreIndex.from_documents(documents, service_context=service_context)
        # store it for later
        index.storage_context.persist("./storage/" + embedding_model_name)
        storage_context = None
    else:
        # load the existing index from the specific embedding model
        storage_context = StorageContext.from_defaults(persist_dir="./storage/" + embedding_model_name)
        index = load_index_from_storage(storage_context)

    query_engine = index.as_query_engine()

    return query_engine

In [ ]:
app2 = load_llamaindex_app()

tru_app2 = tru.Llama(
    app2,
    app_id="LlamaIndex_App1",
    initial_app_loader=load_llamaindex_app
)

In [ ]:
from trulens_eval.schema import AppDefinition

for app_json in AppDefinition.get_loadable_apps():
    print(app_json['app_id'])

LlamaIndex_App1


## Build and pass evaluation questions to the Index

eval_questions = []
with open('eval_questions.txt', 'r') as file:
    for line in file:
        # Remove newline character and convert to integer
        item = line.strip()
        eval_questions.append(item)


for question in eval_questions:
    with tru_recorder as recording:
        sentence_window_engine.query(question)

